## Introducing Opacus

![](https://opacus.ai/img/opacus_logo_vertical.svg)

[Opacus](https://opacus.ai/) is the framework in the PyTorch ecosystem that brings **Differential Privacy** to (PyTorch) Model Training


(from [Introducing Opacus](https://ai.facebook.com/blog/introducing-opacus-a-high-speed-library-for-training-pytorch-models-with-differential-privacy/))

 
> `Opacus` (_is_) a new high-speed library for training PyTorch models with differential privacy (DP) that’s more scalable than existing state-of-the-art methods. 
>
> Differential privacy is a mathematically rigorous framework for quantifying the anonymization of sensitive data. 
>
> It’s often used in analytics, with growing interest in the machine learning (ML) community. 

# Differential Privacy

## Definition

Like $k$-Anonymity, *differential privacy*[3](#fn3) is a **formal notion of privacy** 
(i.e. it's possible to prove that a data release has the property). 

Unlike $k$-Anonymity, however, **differential privacy** is a property of *algorithms*, and not a property of *data*. 

That is, we can prove a *dataset* satisfies differential privacy by proving that an *algorithm* satisfies differential privacy.

> **Definition**:
>
> A function which satisfies differential privacy is often called a *mechanism*. 
> We say that a *mechanism* $F$ satisfies differential privacy if for all *neighboring datasets* $x$ and $x'$, 
> and all possible outputs $S$,
>

\begin{equation}
\frac{\mathsf{Pr}[F(x) = S]}{\mathsf{Pr}[F(x') = S]} \leq e^\epsilon
\end{equation}

**1. Neighbours Datasets**:

Two datasets are considered **neighbours** if they differ in the data by **one single individual**.

**2. $F$ Randomised Function**:

Note that $F$ is typically a *randomised* function, so that the probability distribution describing its outputs is not just a point distribution.

The important implication of this definition is that $F$'s output will be pretty much the same, *with or without* the data of any specific individual.

In other words, the randomness built into $F$ should be "enough" so that an observed output from $F$ will not reveal which of $x$ or $x'$ was the input.

Imagine that my data is present in $x$ but not in $x'$.

**3. The Privacy Budget: $\epsilon$**:

If an adversary can't determine which of $x$ or $x'$ was the input to $F$, then the adversary can't tell whether or not my data was *present* in the input - let alone the contents of that data.

The $\epsilon$ parameter in the definition is called the *privacy parameter* or the *privacy budget*.

$\epsilon$ provides a knob to tune the **amount of privacy** the definition provides.

Small values of $\epsilon$ require $F$ to provide *very* similar outputs when given similar inputs, and therefore provide **higher levels** of privacy.

Large values of $\epsilon$ allow less similarity in the outputs, and therefore provide **less privacy**.


- Small values $\epsilon \rightarrow$ High Privacy
- Large values $\epsilon \rightarrow$ Less Privacy

How should we set $\epsilon$ to prevent bad outcomes in practice? **Nobody knows** (i.e. Open Research Question). 

The general consensus is that $\epsilon$ should be around `1` or smaller, and values of $\epsilon$ above `10` probably don't do much to protect privacy - but this rule of thumb could turn out to be very conservative. 

<span id="fn3">**[3]**: Dwork, C; _Differential Privacy_ in Proceedings of the 33rd International Conference on Automata, Languages and Programming - Volume Part II, 2006 [link](https://doi.org/10.1007/11787006_1)</span>

# Properties of Differential Privacy

In this notebook we will mention three important properties of **differentially private mechanisms** that arise from the definition
 of differential privacy.

These properties are mentioned as they will be used / referenced when we will start generalising DP applications 
to Machine Learning algorithms.


These three properties are:

1. Sequential composition
2. Parallel composition
3. Post processing

## Sequential Composition

The first major property of differential privacy is *sequential composition*, which **bounds** the total 
privacy cost of releasing multiple results of differentially private mechanisms **on the same input data**. 

Formally, the sequential composition theorem for differential privacy says that:

- If $F_1(x)$ satisfies $\epsilon_1$-differential privacy
- And $F_2(x)$ satisfies $\epsilon_2$-differential privacy
- Then the mechanism $G(x) = (F_1(x), F_2(x))$ which releases both results satisfies $\epsilon_1+\epsilon_2$-differential privacy


Sequential composition is a vital property of differential privacy because it enables the design of algorithms that consult the data more than once. 

Sequential composition is also important when multiple separate analyses are performed on a single dataset, since it allows individuals to bound the *total* privacy cost they incur by participating in all of these analyses.

The bound on privacy cost given by sequential composition is an *upper* bound - the actual privacy cost of two particular differentially private releases may be smaller than this, but never larger.

The principle that the $\epsilon$-s "add up" makes sense if we examine the distribution of outputs from a mechanism which averages two differentially private results together.

However, please bear in mind that Sequential composition does not provide an **exact** upper bound: the exact total privacy cost can be indeed lower than the upper bound!.

## Parallel Composition

The second important property of differential privacy is called *parallel composition*. 

Parallel composition can be seen as an alternative to sequential composition - a second way to calculate a bound on the total privacy cost of multiple data releases. 

Parallel composition is based on the idea of **splitting** your dataset into disjoint chunks and running a 
differentially private mechanism on each chunk separately. 

Since the chunks are **disjoint**, each individual's data appears in *exactly* one chunk - so even if there are $k$ chunks in total (and therefore $k$ runs of the mechanism), the mechanism runs exactly once on the data of each *individual*. 

Formally,
 - If $F(x)$ satisfies $\epsilon$-differential privacy
 - And we split a dataset $X$ into $k$ disjoint chunks such that $x_1 \cup ... \cup x_k = X$
 - Then the mechanism which releases all of the results $F(x_1), ..., F(x_k)$ satisfies $\epsilon$-differential privacy

Note that this is a much better bound than sequential composition would give. 

Since we run $F$ $k$ times, sequential composition would say that this procedure satisfies $k\epsilon$-differential privacy. 

Parallel composition allows us to say that the total privacy cost is just $\epsilon$.

The formal definition matches up with our intuition - if each participant in the dataset contributes one row to $X$, then this row will appear in *exactly* one of the chunks $x_1, ..., x_k$. 

That means $F$ will only "see" this participant's data *one time*, meaning a privacy cost of $\epsilon$ is appropriate for that individual. Since this property holds for all individuals, the privacy cost is $\epsilon$ for everyone.

## Post-processing

The third property of differential privacy is called *post-processing*. 

The idea is simple: it's impossible to **reverse the privacy protection** provided by differential privacy by post-processing the data in some way. 

Formally:

- If $F(X)$ satisfies $\epsilon$-differential privacy
- Then for any (deterministic or randomized) function $g$, $g(F(X))$ satisfies $\epsilon$-differential privacy

The post-processing property means that it's always safe to perform arbitrary computations on the output of a differentially private mechanism - there's no danger of reversing the privacy protection the mechanism has provided. 

In particular, it's fine to perform post-processing that might reduce the noise or improve the signal in the mechanism's output (e.g. replacing negative results with zeros, for queries that shouldn't return negative results). 

The other implication of the **post-processing** property is that differential privacy provides resistance against privacy attacks based on **auxiliary information**. 

For example, the function $g$ might contain auxiliary information about elements of the dataset, and attempt to perform a linkage attack using this information. The post-processing property says that such an attack is limited in its effectiveness by the privacy parameter $\epsilon$, regardless of the auxiliary information contained in $g$.

## The Laplace Mechanism

Differential privacy is typically used to answer specific queries. Let's consider a query on the census data, *without* differential privacy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

In [2]:
DATASET_URL = "https://raw.githubusercontent.com/uvm-plaid/programming-dp/master/notebooks/adult_with_pii.csv"
adult = pd.read_csv(DATASET_URL)

**Q** "How many individuals in the dataset are 40 years old or older?"

This is an example of a **Count Query**.

In [3]:
adult[adult['Age'] >= 40].shape[0]

14237

## Laplace Mechanism

The easiest way to achieve differential privacy for this query is to add **random noise to its answer**. 

The key challenge is to add enough noise to satisfy the definition of differential privacy, but not so much that the answer becomes too noisy to be useful. 

To make this process easier, some basic *mechanisms* have been developed in the field of differential privacy, which describe exactly what kind of - and how much - noise to use. 

One of these is called the *Laplace mechanism*[4](#fn4).

> **Definition**
> 
>According to the Laplace mechanism, for a function $f(x)$ which returns a number, the following definition of $F(x)$ satisfies $\epsilon$-differential privacy:
>
>\begin{equation}
F(x) = f(x) + \textsf{Lap}(\frac{s}{\epsilon})
\end{equation}
>
>where $s$ is the *sensitivity* of $f$, and $\textsf{Lap}(S)$ denotes sampling from the Laplace distribution with center 0 and scale $S$.


**Sensitivity**:

The *sensitivity* of a function $f$ is the amount $f$'s output changes when its input changes by 1. 

Sensitivity is a complex topic, and an integral part of designing differentially private algorithms. 

Let's just point out that *counting queries* always have a sensitivity of `1`: if a query counts the number of rows in the dataset with a particular property, and then we modify exactly one row of the dataset, then the query's output can change by at most `1`.

Thus we can achieve differential privacy for our example query by using the `Laplace mechanism` with `sensitivity=1` and an $\epsilon$ of our choosing.

For now, let's pick $\epsilon = 0.1$. We can sample from the Laplace distribution using Numpy's `random.laplace`.

<span id="fn4">**[4]**: Dwork, C.; _Calibrating Noise to Sensitivity in Private Data Analysis_ in Proceedings of the Third Conference on Theory of Cryptography, 2006 [link](https://doi.org/10.1007/11681878_14)</span>

In [4]:
np.random.seed(12345)

sensitivity = 1
epsilon = 0.1

adult[adult['Age'] >= 40].shape[0] + np.random.laplace(loc=0, scale=sensitivity/epsilon)

14256.606434525418

You can see the effect of the noise by running this code multiple times. Each time, the output changes, but most of the time, the answer is close enough to the true answer (14,235) to be useful.

In [5]:
true_count_stat = adult[adult['Age'] >= 40].shape[0]
Lap = np.random.laplace(loc=0, scale=sensitivity/epsilon, size=30)
print(f"True Count Statistic: {true_count_stat}")
for i in range(30):
    print(f"{i}) {(true_count_stat + Lap[i]):0.2f}")

True Count Statistic: 14237
0) 14232.42
1) 14227.00
2) 14228.06
3) 14238.46
4) 14239.12
5) 14263.45
6) 14240.66
7) 14243.89
8) 14240.67
9) 14243.84
10) 14262.59
11) 14196.12
12) 14221.53
13) 14231.85
14) 14240.75
15) 14246.67
16) 14250.64
17) 14263.49
18) 14242.93
19) 14240.35
20) 14242.71
21) 14236.33
22) 14232.71
23) 14235.71
24) 14243.15
25) 14281.25
26) 14241.37
27) 14245.71
28) 14226.27
29) 14207.76


## How Much Noise is Enough?

How do we know that the Laplace mechanism adds enough noise to prevent the re-identification of individuals in the dataset? 

For one thing, we can try to break it!

Let's write down a **malicious counting query**, which is specifically designed to determine whether Karrie Trusslove has an income greater than `$50k`.

In [6]:
karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0]

1

This result definitely violates Karrie's privacy, since it reveals the value of the income column for Karrie's row.

Since we know how to ensure differential privacy for counting queries with the Laplace mechanism, we can do so for this query:

In [7]:
sensitivity = 1
epsilon = 0.1

karries_row = adult[adult['Name'] == 'Karrie Trusslove']
karries_row[karries_row['Target'] == '<=50K'].shape[0] + np.random.laplace(loc=0, scale=sensitivity/epsilon)

10.181436671261721

In [8]:
query = karries_row[karries_row['Target'] == '<=50K'].shape[0]

In [9]:
F = query + np.random.laplace(loc=0, scale=sensitivity/epsilon)

np.mean([F for _ in range(100)])

17.47373848723544

Is this the true answer ?

There's too much noise to be able to reliably tell.

This is how differential privacy is *intended* to work - the approach does not *reject* queries which are determined to be malicious; instead, it adds enough noise that the results of a malicious query will be useless to the adversary.